In [4]:
import pandas as pd
import requests as r
import json

In [9]:
#setting konfigurasi
with open("env.json", "r", encoding="utf-8") as file:
    config = json.load(file)  # Membaca dan parsing JSON


In [11]:
#Get Daftar Data Dinamis BPS Bali
def getVarByDomain(domain='5100'):
    global config
    var_pages=[]
    var = r.get(f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/key/{config['token']}/')
    pages=var.json()['data'][0]['pages']
    current_page = 1
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/page/{current_page}/key/{config['token']}/'
        var_page=r.get(url)
        var_pages.append(pd.json_normalize(var_page.json()['data'][1]))
        current_page=current_page+1
    var_pages_df = pd.concat(var_pages)
    return var_pages_df

In [37]:
#Get Daftar Periode
def getPriodeByVar(domain='5100',varId='106'):
    global config
    current_page = 1
    url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
    dt_json = r.get(url)
    pages = dt_json.json()['data'][0]['pages']
    dt_all = []
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
        dt = pd.json_normalize(dt_json.json()['data'][1])
        dt_all.append(dt)
        current_page = current_page+1
    dt_all = pd.concat(dt_all)
    dt_all['varId']=varId
    return dt_all

In [59]:
def crossJoin(left,right):
    if left.empty:
        left = right
    else:
        left = pd.merge(left,right,how='cross')
    return left
def getDataByVarId(varId='106',domain='5100'):
    global config
    #Banyaknya Wisatawan Mancanegara Bulanan ke Bali Menurut Pintu Masuk
    dt_th = getPriodeByVar(domain=domain,varId=varId)
    dt_all = []
    for i in dt_th.index:
        th_id = dt_th.iloc[i]['th_id']
        url = f'https://webapi.bps.go.id/v1/api/list/model/data/domain/{domain}/var/{varId}/key/{config['token']}/th/{th_id}/'
        dt_json = r.get(url)
        attr = pd.DataFrame()
        if 'vervar' in dt_json.json().keys():
            vervar=pd.json_normalize(dt_json.json()['vervar'])
            vervar=vervar.rename(columns={'val': 'verval_val','label': 'vervar_label'})
            attr = crossJoin(attr,vervar)
        if 'var' in dt_json.json().keys():
            var=pd.json_normalize(dt_json.json()['var'])
            var=var.rename(columns={'val': 'var_val','label': 'var_label'})
            var=var[['var_val']]
            attr = crossJoin(attr,var)
        if 'turvar' in dt_json.json().keys():
            turvar=pd.json_normalize(dt_json.json()['turvar'])
            turvar=turvar.rename(columns={'val': 'turvar_val','label': 'turvar_label'})
            attr = crossJoin(attr,turvar)
        if 'tahun' in dt_json.json().keys():
            tahun=pd.json_normalize(dt_json.json()['tahun'])
            tahun=tahun.rename(columns={'val': 'tahun_val','label': 'tahun_label'})
            attr = crossJoin(attr,tahun)
        if 'turtahun' in dt_json.json().keys():
            turtahun=pd.json_normalize(dt_json.json()['turtahun'])
            turtahun=turtahun.rename(columns={'val': 'turtahun_val','label': 'turtahun_label'})
            attr = crossJoin(attr,turtahun)
        val_columns = [col for col in attr.columns if col.endswith('val')]
        attr['id']=attr[val_columns].astype('str').agg(''.join, axis=1)
        dt_content=pd.json_normalize(dt_json.json()['datacontent']).transpose().reset_index().rename(columns={'index':'content_val',0:'content_label'})
        attr = pd.merge(attr,dt_content,left_on='id',right_on='content_val',how='left')
        attr = attr[attr.content_label.notna()]
        dt_all.append(attr)
    dt_all=pd.concat(dt_all)
    return dt_all

In [ ]:
daftar_data=getVarByDomain(domain='5100')
#daftar_data.to_excel('daftar_data_web_api_dinamis.xlsx')
#Contoh akan mengambil data by id dari daftar data
data=getDataByVarId(domain='5100',varId='47')

In [ ]:
### id 47 Inflasi Tahunan Kota Singaraja Menurut 7 Kelompok Pengeluaran
### korelasi dengan web BPS
### modifikasi ke bentuk seperti ini 
# https://bali.bps.go.id/id/statistics-table/2/NDcjMg==/inflasi-tahunan-kota-singaraja-menurut-7-kelompok-pengeluaran.html

In [63]:
data.to_excel('data_47.xlsx')